In [1]:
using Pkg
using Revise
Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
# Constants and Parameters
const n, a, b = 3, 1, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Rastringin # Objective function

  Activating project at `~/globtim`


Rastringin (generic function with 1 method)

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [1]:
rand_center = [0.0, 0.0, 0,];
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

UndefVarError: UndefVarError: `n` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [ ]:
# df_cheb = solve_and_parse(pol_cheb, x, f, TR)
pts_cheb = solve_polynomial_system(x, TR.dim, d, pol_cheb.coeffs; basis=:chebyshev)
df_cheb = process_crit_pts(pts_cheb, f, TR)
sort!(df_cheb, :z, rev=false)

Dimension m of the vector space: 286
Tracking 729 paths...   0%|▏                            |  ETA: 0:34:16



Tracking 729 paths... 100%|█████████████████████████████| Time: 0:00:06
  # paths tracked:                  729
  # non-singular solutions (real):  729 (125)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         729 (125)

=== Starting Critical Points Processing (dimension: 3) ===
Processed 125 points (0.077s)


Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,-1.07878e-17,-1.13714e-16,1.90256e-17,-30.0
2,-1.53547e-17,-0.991184,5.43618e-16,-29.0022
3,-1.20943e-17,-1.5946e-16,0.991184,-29.0022
4,-5.65745e-18,0.991184,2.6312e-16,-29.0022
5,0.991184,-3.88227e-17,4.47476e-18,-29.0022
6,-0.991184,-3.92811e-17,1.0711e-17,-29.0022
7,-1.08507e-17,-2.97316e-16,-0.991184,-29.0022
8,0.991184,0.991184,2.27085e-16,-28.0044
9,0.991184,-0.991184,5.15841e-16,-28.0044


In [4]:
using GLMakie

In [5]:
grid = scale_factor * generate_grid(3, 100)  # 3D grid
values = map(f, grid); # Prepare level set data for specific level

In [6]:
fig = create_level_set_visualization(f, grid, df_cheb, (-30.0, 30.))

In [7]:
GLMakie.closeall()